In [2]:
import os
import gzip
from struct import Struct
import pandas as pd
import time


matrix_path = './alevin_matrices/'
output_path = './alevin_output/'



In [3]:
def get_cb_count():
    cb_count = dict()
    for dir_name in os.listdir(matrix_path):
        if dir_name[0] != '.':
            print(dir_name)
            f = open(matrix_path + dir_name + "/"+dir_name+'_rows.txt', 'r')
            for line in f:
                line = line.strip()
                cb_count[line] = cb_count.get(line, 0)+1
            f.close()
    return cb_count

cb_count = get_cb_count()


E18_20160930_Neurons_Sample_01_quants_mat


In [6]:
from collections import defaultdict
def merge():
    start = time.time()
    values_file = open(output_path + 'concatenated.bin.gz', 'wb')
    rows_file = open(output_path + 'concatenated_rows.txt', 'w')

    common_cb = defaultdict(list)

    for dir_name in os.listdir(matrix_path):
        if dir_name[0] != '.':

            gene_file = matrix_path + dir_name + "/"+dir_name+'_cols.txt'
            print('gene file', gene_file)
            gene_names = pd.read_table(gene_file, header=None)[0].values
            
            num_genes = len(gene_names)
            header_struct = Struct( "d" * num_genes)
            
            rows = open(matrix_path + dir_name + '/'+dir_name+'_rows.txt', 'r')
            matrix = gzip.open(matrix_path + dir_name + '/'+dir_name+'.gz', 'r')
            
            
            count = 0
            while True:
                count += 1
                try:
                    cell_values = header_struct.unpack_from(matrix.read(header_struct.size))
                except:
                    print('counted: ', count)
                    print('Error in unpack')
                    break
                #print(cell_values)
                cb = next(rows).strip()
                if cb_count[cb] == 1:
                    values_file.write(header_struct.pack(*cell_values))
                    rows_file.write(cb + '\n')
                else:
                    #common_cb[cb].append(cell_values) first version (182 secs)
                    #values_file.write(str(cell_values) + '\n')  ## second version (100 sec)
                    values_file.write(header_struct.pack(*cell_values)) ##current version (84.secs)
                    rows_file.write(cb+"_"+str(cb_count[cb])+ '\n')
                    cb_count[cb] -= 1
                    
            rows.close()
            
            matrix.close()
    """ ### ideally was done to keep same rows together for easier compression. 
    for key, val in common_cb.items():
            index = 0
            for cell_values in val:
                rows_file.write(key + '_' + str(index) + '\n')
                values_file.write(str(cell_values) + '\n')
                index += 1
    """
    values_file.close()
    rows_file.close()
    end = time.time()
    print('Finished. Time: ', end - start)

merge()

gene file ./alevin_matrices/E18_20160930_Neurons_Sample_01_quants_mat/E18_20160930_Neurons_Sample_01_quants_mat_cols.txt
counted:  12013
Error in unpack
Finished. Time:  84.14836931228638
